In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd


from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D,GRU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Concatenate
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
import json
import numpy as np
from sklearn.model_selection import train_test_split
import sys
sys.path.append(r"/content/drive/MyDrive/khoa luan/mebe_domain_pre")

from modules.evaluate import cal_aspect_prf



In [ ]:
max_len=50
feat_vec=300
with open ("/content/drive/MyDrive/khoa luan/mebe_domain_pre/encode_data_shopee.txt", "r") as f:
    encode_data_shopee = json.loads(f.read())
with open ("/content/drive/MyDrive/khoa luan/mebe_domain_pre/output_shopee.txt", "r") as f:
    output_ori_shopee = np.array(json.loads(f.read()))
    
padded_doc_shopee=pad_sequences(encode_data_shopee,maxlen=max_len,padding="post")


with open ("/content/drive/MyDrive/khoa luan/mebe_domain_pre/encode_data_tiki.txt", "r") as f:
    encode_data_tiki = json.loads(f.read())
padded_doc_tiki=pad_sequences(encode_data_tiki,maxlen=max_len,padding="post")
with open ("/content/drive/MyDrive/khoa luan/mebe_domain_pre/output_tiki.txt", "r") as f:
    output_ori_tiki = np.array(json.loads(f.read()))
print(padded_doc_tiki.shape, output_ori_tiki.shape)
X_train_tiki, X_test_tiki, y_train_ori_tiki, y_test_ori_tiki = train_test_split(padded_doc_tiki, output_ori_tiki, test_size=0.2, random_state=14)
X_train_aug = np.concatenate((X_train_tiki, padded_doc_shopee))
y_train_ori_aug = np.concatenate((y_train_ori_tiki, output_ori_shopee)) 


In [ ]:
# len(y_train_ori_tiki[:,0])
# y_train_tiki = y_train_ori_tiki[:, [0, 4]]
# y_test_tiki = y_test_ori_tiki[:, [0, 4]]

In [ ]:
y_train_tiki = np.delete(y_train_ori_tiki, 6, 1)
y_test_tiki = np.delete(y_test_ori_tiki, 6, 1)
y_train_aug = np.delete(y_train_ori_aug, 6, 1)

In [ ]:
X_train_tiki.shape

In [ ]:
embed_trained = np.load('/content/drive/MyDrive/khoa luan/mebe_domain_pre/modules/aspect/weights/embedding_trained_mat.npy')
lstm_trained = np.load('/content/drive/MyDrive/khoa luan/mebe_domain_pre/modules/aspect/weights/lstm_trained_mat.npy', allow_pickle=True)
# len(lstm_trained)
embed_mat = np.load('/content/drive/MyDrive/khoa luan/mebe_domain_pre/modules/aspect/weights/embedding_mat.npy')
# len(embed_mat)
len(embed_trained[0])
conv_train = np.load('/content/drive/MyDrive/khoa luan/mebe_domain_pre/modules/aspect/weights/convo_trained_mat.npy', allow_pickle=True)


In [ ]:
# convo_trained = np.load('./weights/convo_trained_mat.npy', allow_pickle=True)
# len(convo_trained)

In [ ]:
from sklearn import metrics
def best_thresh(X_test, y_test, model):
#     thresh = np.array([])
    best_score = 0
    best_thr = 0
    y_pre = model.predict(X_test)
    for thr in np.arange(0.1,0.9,0.01):
        score = metrics.f1_score(y_test,(y_pre>thr).astype(int), average='macro')
        if score > best_score: 
            best_score = score
            best_thr = thr
    return best_thr
#         print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(y_test,(y_pre>thresh).astype(int), average='macro')))


In [ ]:


def get_model_distilled(dis_thr):
    inp = Input(shape=(max_len,))
    # x = Embedding(len(embed_mat), feat_vec, weights=[embed_mat], input_length=max_len,trainable=True)(inp) 

    embed_distill = Embedding(len(embed_trained[0]), feat_vec, weights=[embed_trained[0]], input_length=max_len,trainable=False)(inp) # trọng số của từ sẽ không được train lại
    embed_train = Embedding(len(embed_trained[0]), feat_vec, weights=[embed_mat], input_length=max_len,trainable=True)(inp)
    reduce_embed_distill = Dense(dis_thr, activation="relu")(embed_distill)

    embed_concat = Concatenate(axis=-1)([reduce_embed_distill,embed_train])
#     x = Dropout(0.4)(x)

    # embed_combine = dis_thr*embed_distill + embed_train
#     x = Dropout(0.4)(embed_combine)

    # lstm_distill = Bidirectional(LSTM(128, return_sequences=True))(x)
    # reduce_lstm_distill = Dense(dis_thr, activation="relu")(lstm_distill)
    # lstm_train = Bidirectional(LSTM(128, return_sequences=True))(x)
    # lstm_concat = Concatenate(axis=-1)([reduce_lstm_distill,lstm_train])
    # lstm_combine = dis_thr*lstm_distill+lstm_train
    x = Bidirectional(LSTM(128, return_sequences=True), trainable=True)(embed_concat)
    x = Conv1D(64,3,activation="relu")(x)  
    x = GlobalMaxPool1D()(x)
#     convo_distill = Conv1D(64,3,activation="relu")(lstm_combine)    
#     convo_train = Conv1D(64,3,activation="relu")(lstm_combine)
#     convo_concat = Concatenate(axis=-1)([0.2*convo_distill,0.8*convo_train])
#     convo_combine = 0.3*convo_distill+0.7*convo_train
    # conv_distill = Conv1D(64,3,activation="relu")(x)
    # maxpool_distill = GlobalMaxPool1D()(conv_distill)
    # reduce_maxpool_distill = Dense(dis_thr, activation="relu")(maxpool_distill)
    # conv_train = Conv1D(64,3,activation="relu")(x)
    # maxpool_train = GlobalMaxPool1D()(conv_train)
#     x = Dropout(0.4)(x)
    # maxpool_concat = Concatenate(axis=-1)([reduce_maxpool_distill,maxpool_train])
    # maxpool_combine = dis_thr*maxpool_distill+maxpool_train

    x = Dense(64, activation="relu")(x)
#     x = Dense(32, activation="relu")(x)
    x = Dropout(0.35)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    print(model.layers[2].name)
    # model.layers[2].trainable = False
    #     model.layers[11].trainable = False
#     print(model.layers[6].trainable)
    print(model.summary())
    f1 = tfa.metrics.F1Score(num_classes=6, average='macro', threshold=0.5)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.005), metrics= [f1])
    return model
def get_model_normal():
    inp = Input(shape=(max_len,))
    x = Embedding(len(embed_mat), feat_vec, weights=[embed_mat], input_length=max_len,trainable=True)(inp) # trọng số của từ sẽ không được train lại
    x = Bidirectional(LSTM(128, return_sequences=True), trainable=True)(x)
    x = Conv1D(64,3,activation="relu")(x)

#     convo_distill = Conv1D(64,3,activation="relu")(x)
#     convo_train = Conv1D(64,3,activation="relu")(x)
#     convo_concat = Concatenate(axis=-1)([0.2*convo_distill,0.8*convo_train])
    x = GlobalMaxPool1D()(x)
    x = Dense(128, activation="relu")(x)
#     x = Dense(32, activation="relu")(x)
#     x = Dropout(0.5)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    print(model.summary())
    f1 = tfa.metrics.F1Score(num_classes=6, average='macro', threshold=0.5)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.005), metrics=[f1])
    return model

In [ ]:
def run_model(dis_thr):#):

    # rlp = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_f1_score',mode='max', patience=3, verbose=1, factor=0.4)
    es = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score',mode='max', patience=10, verbose=1, restore_best_weights=True)
    filepath = ('/content/drive/MyDrive/khoa luan/mebe_domain_pre/modules/aspect/weights/best_student.hdf5')
    checkpoint = ModelCheckpoint(filepath, monitor='val_f1_score', verbose=1, save_best_only=True,save_weights_only=True, mode='max')

    # model = get_model_distilled(dis_thr)
    model = get_model_distilled(dis_thr)
    # model.layers[2].set_weights(lstm_trained)
    model.fit(X_train_tiki, y_train_tiki, batch_size=64, epochs=20, validation_data=(X_test_tiki, y_test_tiki), callbacks=[checkpoint, es])
    # new_model = get_model_distilled(dis_thr)
    new_model = get_model_distilled(dis_thr)
    new_model.load_weights('/content/drive/MyDrive/khoa luan/mebe_domain_pre/modules/aspect/weights/best_student.hdf5')
    y_pre = new_model.predict(X_test_tiki)
    best_thr = best_thresh(X_test_tiki, y_test_tiki, new_model)
    y_pre_round = []
    for i in range(len(y_pre)):
        y_pre_round.append(1*(y_pre[i] >= best_thr))
        y_pre_round[i] = y_pre_round[i].tolist()
    X = cal_aspect_prf(y_test_ori_tiki.tolist(), y_pre_round, num_of_aspect=6, verbal=True)
    return X

In [ ]:
# thresh = np.array([])
# for thr in np.arange(0.1,0.9,0.01):
#     thresh = np.append(thresh, metrics.f1_score(y_test_tiki,(y_pre>thr).astype(int), average='macro'))
#     print(metrics.f1_score(y_test_tiki,(y_pre>thr).astype(int), average='macro'))
# max(thresh)

In [ ]:
# dis_thresh =[6, 13, 19, 26, 32, 38, 45, 51, 58, 64]
# dis_thresh = [26, 51, 77, 102, 128, 154, 179, 205, 230, 256]
dis_thresh = [30, 60, 90, 120, 150, 180, 210, 240, 270, 300]
# dis_thresh = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
# dis_thresh =[300]
for dis_thr in dis_thresh:    
  sum_p = [0]*6
  sum_r = [0]*6
  sum_f1 = [0]*6
  sum_micro = [0]*3
  sum_macro = [0]*3
  count = 0

  for i in range (20):   
      count += 1
      res = run_model(dis_thr)
      sum_p = [(x + y) for x, y in zip(res[0], sum_p)]
      sum_r = [(x + y) for x, y in zip(res[1], sum_r)]
      sum_f1 = [(x + y) for x, y in zip(res[2], sum_f1)]
      sum_micro = [(x + y) for x, y in zip(res[3], sum_micro)]
      sum_macro = [(x + y) for x, y in zip(res[4], sum_macro)]
      cur_macro = [(x/count) for x in sum_macro]
      print(cur_macro)
      print(count)
      print(dis_thr)   
  final_p = [(x/count) for x in sum_p]
  final_r = [(x/count) for x in sum_r]
  final_f1 = [(x/count) for x in sum_f1]
  final_micro = [(x/count) for x in sum_micro]
  final_macro = [(x/count) for x in sum_macro]
  label = ['ship','giá','chính hãng''','chất lượng','dịch vụ','an toàn']
  res = {'label':label,'p':final_p, 'r':final_r, 'f1':final_f1}
  df = pd.DataFrame(res)
  df1 = pd.DataFrame({'micro':final_micro, 'macro': final_macro})
  final_df=pd.concat([df,df1], axis=1)
  final_df.to_csv(r"/content/drive/MyDrive/khoa luan/mebe_domain_pre/ket qua/distill embed(300 train x distill)/20 times/{} distill.csv".format(dis_thr), encoding='utf-8-sig')

    

In [ ]:
# label = ['ship','giá','chính hãng''','chất lượng','dịch vụ','an toàn']
# res = {'label':label,'p':sum_p, 'r':sum_r, 'f1':sum_f1}
# df = pd.DataFrame(res)
# df1 = pd.DataFrame({'micro':sum_micro, 'macro': sum_macro})
# final_df=pd.concat([df,df1], axis=1)

# final_df.to_csv(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain_pre\ket qua\distill embed(300 train x distill)\20 times\300 train 30 distill.csv", encoding='utf-8-sig')

In [ ]:
# model = get_model_distilled()
# .fit(X_train_tiki, y_train_tiki, batch_size=8, epochs=15, validation_data=(X_test_tiki, y_test_tiki), callbacks=[checkpoint])



In [ ]:
# model.layers[3].set_weights(conv_train)
# model.layers[11].set_weights(convo_trained)
# model.layers[3].set_weights(convo_trained)

In [ ]:
# model.fit(X_train_tiki, y_train_tiki, batch_size=64, epochs=20, validation_data=(X_test_tiki, y_test_tiki), callbacks=[checkpoint, es])


In [ ]:
# new_model = get_model_distilled()
# new_model.load_weights('./weights/best_student.hdf5')


In [ ]:
sum_p = list(range(6))
sum_p

In [ ]:
# type(new_model)

In [ ]:
# y_pre=evaluate(X_test_tiki, y_test_tiki, new_model)

In [ ]:
# thresh=0.37


# y_pre_round = []
# for i in range(len(y_pre)):
#     y_pre_round.append(1*(y_pre[i] >= thresh))
#     y_pre_round[i] = y_pre_round[i].tolist()
    




In [ ]:
# label = ['ship','giá','chính hãng''','chất lượng','dịch vụ','an toàn']
# res = {'label':label,'p':X[0], 'r':X[1], 'f1':X[2]}
# df = pd.DataFrame(res)
# df1 = pd.DataFrame({'micro':X[3], 'macro': X[4]})
# final_df=pd.concat([df,df1], axis=1)

# final_df.to_csv(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain_pre\ket qua\distill convo (64 train x distill)\64 train 64 distill.csv", encoding='utf-8-sig')


In [ ]:
a1 = (1, 2, 3)
a2 = list(range(3))
a = [(x + y)/count for x, y in zip(a1, a2)]
a

In [ ]:
a3/3

In [ ]:
for i in range (20):

    count += 1
    print(count)

In [ ]:
arr = np.array([1,2,3])
np.append(arr,4)
arr

In [ ]:
a = [0]*3

In [ ]:
a